In [1]:
# Imports
import os
import re
import requests
import webbrowser
from datetime import datetime
from dotenv import load_dotenv
import pandas as pd
from stravalib.client import Client

In [5]:
# Constants
ACTIVITY_STREAM_TYPES = [
    'time',
    'latlng',
    'distance',
    'altitude',
    'velocity_smooth',
    'heartrate',
    'grade_smooth'
]

In [13]:
# Init Strava client
client = Client()

# Load environment variables
load_dotenv('../../.env')
STRAVA_CLIENT_ID = os.getenv('SOCIAL_AUTH_STRAVA_KEY')
STRAVA_CLIENT_SECRET = os.getenv('SOCIAL_AUTH_STRAVA_SECRET')

In [14]:
# get authorization url for first time user manual authorization for access 
# to their strava account
auth_url = client.authorization_url(client_id=STRAVA_CLIENT_ID, 
    redirect_uri='http://127.0.0.1:8000/', # for local development
    scope=['read_all','profile:read_all','activity:read_all'])

# Authenticate and add code to environment variables TODO: automate this
webbrowser.open(auth_url, new=0, autoraise=True)

True

In [17]:
# Load new env with code
load_dotenv('../../.env', override=True)

# Get code from the authorization link
STRAVA_CLIENT_CODE = os.getenv('SOCIAL_AUTH_STRAVA_CODE')
STRAVA_CLIENT_CODE

'1e0954eada511baa1c6c89c0131b8648c6076dfa'

In [38]:
# get test data about athlete
current_athlete = client.get_athlete()
athlete_name = (current_athlete.firstname + '_' + current_athlete.lastname).lower()

# Create folder to store data for that athlete
folder_path = f'../data/{athlete_name}'
if(not os.path.exists(folder_path)):
    print('Creating new athlete folder...')
    os.mkdir(folder_path)

Creating new athlete folder...


In [39]:
# Define dates for query. Note: there is a Strava API rate limit for given 15 minute period.
date_start = datetime(2020, 1, 1)
date_end = datetime.now()
# Get all activities
activities = client.get_activities(before=date_end, after=date_start)

In [40]:
# Save running activities with heart rate data as csv files
for a in activities:
    if(a.type == 'Run' and a.has_heartrate):
        # Get dictionary of activity streams
        activity_stream = client.get_activity_streams(
            activity_id=a.id, types=ACTIVITY_STREAM_TYPES, resolution='medium')
        # Extract streams of interest
        data = {key: activity_stream.get(key).data for key in ACTIVITY_STREAM_TYPES}
        # Create dataframe from streams
        df = pd.DataFrame(data=data)
        # Create file name from GMT start date and formatted activity name
        filename = a.start_date.strftime('%Y-%m-%dT%H.%M.%S') + 'GMT_' + re.sub('[^a-zA-Z]', '', a.name) 
        print(filename)
        # Save dataframe to csv file
        df.to_csv(f'../data/{athlete_name}/{filename}.csv', index=False)

2023-01-28T15.30.11GMT_Lindseyranak
2023-01-24T23.48.08GMT_EveningRun
2023-01-24T23.09.20GMT_EveningRun
2023-01-15T16.07.50GMT_MorningRun
2022-12-27T19.04.52GMT_AfternoonRun
2022-12-10T14.05.02GMT_MorningRun
2022-12-09T21.28.25GMT_AfternoonRun
2022-12-07T15.17.49GMT_MorningRun
2022-12-06T00.52.40GMT_AfternoonRun
2022-12-03T17.14.47GMT_MorningRun
2022-12-02T23.11.31GMT_AfternoonRun
2022-11-28T23.15.04GMT_AfternoonRun
2022-11-27T16.18.33GMT_MorningRun
2022-11-25T18.41.28GMT_LunchRun
2022-11-17T23.42.41GMT_AfternoonRun
2022-11-13T15.56.27GMT_Teapotlongrun
2022-11-11T23.20.28GMT_AfternoonRun
2022-11-10T14.24.34GMT_MorningRun
2022-11-08T23.20.19GMT_AfternoonRun
2022-11-06T16.02.55GMT_MorningRun
2022-11-05T22.10.22GMT_AfternoonRun
2022-11-04T00.42.22GMT_AfternoonRun
2022-11-02T01.21.04GMT_EveningRun
2022-11-02T00.37.17GMT_WorkoutWarmUp
2022-10-30T15.19.37GMT_MorningRun
2022-10-29T15.41.05GMT_MorningRun
2022-10-27T12.43.20GMT_Wogtempo
2022-10-25T13.51.00GMT_MorningRun
2022-10-23T15.46.57GMT_C